In [1]:
import sys
sys.path.append("../../")  # access to local modules

In [2]:
import os
import tensorflow as tf
import keras_tuner as kt
import numpy as np
import pandas as pd

2025-03-28 09:45:09.077085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
LOGS_PATH: str = "logs/"
VALIDATION_NUMBER: int = 5000
CLASS_NAMES = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(x_train_full, y_train_full), (x_test, y_test) = fashion_mnist
x_train, y_train = x_train_full[:-VALIDATION_NUMBER], y_train_full[:-VALIDATION_NUMBER]
x_valid, y_valid = x_train_full[-VALIDATION_NUMBER:], y_train_full[-VALIDATION_NUMBER:]
x_train, x_valid, x_test = x_train / 255.0, x_valid / 255.0, x_test / 255.0

In [5]:
pos_class_id = CLASS_NAMES.index("Pullover")
neg_class_id = CLASS_NAMES.index("T-shirt/top")

def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))

In [6]:
x_train[0].shape

(28, 28)

In [7]:
tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=x_train[0].shape),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=["accuracy"])

/home/dierck/workspace/python/MLTraining/Notebooks/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-03-28 09:45:11.390923: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-28 09:45:13.231834: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-28 09:45:13.231878: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:920] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so return

In [8]:
(x_train_A, y_train_A), (x_train_B, y_train_B) = split_dataset(x_train, y_train)
(x_valid_A, y_valid_A), (x_valid_B, y_valid_B) = split_dataset(x_valid, y_valid)
(x_test_A, y_test_A), (x_test_B, y_test_B) = split_dataset(x_test, y_test)
x_train_B, y_train_B = x_train_B[:200], y_train_B[:200]

In [9]:
history = model_A.fit(x_train_A, y_train_A, epochs=20, validation_data=(x_valid_A, y_valid_A))
model_A.save("my_model_A.keras")

Epoch 1/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 840us/step - accuracy: 0.4493 - loss: 1.5943 - val_accuracy: 0.7696 - val_loss: 0.7414
Epoch 2/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.7819 - loss: 0.6765 - val_accuracy: 0.8263 - val_loss: 0.5410
Epoch 3/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 769us/step - accuracy: 0.8336 - loss: 0.5172 - val_accuracy: 0.8543 - val_loss: 0.4562
Epoch 4/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - accuracy: 0.8585 - loss: 0.4416 - val_accuracy: 0.8669 - val_loss: 0.4081
Epoch 5/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 785us/step - accuracy: 0.8712 - loss: 0.3967 - val_accuracy: 0.8747 - val_loss: 0.3779
Epoch 6/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 778us/step - accuracy: 0.8772 - loss: 0.3677 - val_accuracy: 0.8782 - val_loss: 0.3573
Epoch 7/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - accuracy: 0.8829 - loss: 0.3471 - val_accuracy: 0.8807 - val_loss: 0.3424
Epoch 8/20
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 1s 778us/step - accuracy: 0.8868 -

In [10]:
tf.random.set_seed(42)
model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_B.fit(x_train_B, y_train_B, epochs=20,
                      validation_data=(x_valid_B, y_valid_B))
model_B.evaluate(x_test_B, y_test_B)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4582 - loss: 0.7131 - val_accuracy: 0.5005 - val_loss: 0.6745
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4951 - loss: 0.6823 - val_accuracy: 0.5470 - val_loss: 0.6501
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5626 - loss: 0.6548 - val_accuracy: 0.6281 - val_loss: 0.6280
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6562 - loss: 0.6299 - val_accuracy: 0.6993 - val_loss: 0.6074
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6804 - loss: 0.6070 - val_accuracy: 0.7587 - val_loss: 0.5887
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7568 - loss: 0.5863 - val_accuracy: 0.8012 - val_loss: 0.5717
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8117 - loss: 0.5676 - val_accuracy: 0.8318 - val_loss: 0.5559
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8685 - loss: 0.5502 - val_accuracy: 0.8595 - val_loss: 0.5414
Epoch 

[0.4280036389827728, 0.921999990940094]

In [11]:
model_A = tf.keras.models.load_model("my_model_A.keras")
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [12]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [13]:
history = model_B_on_A.fit(x_train_B, y_train_B, epochs=4, validation_data=(x_valid_B, y_valid_B))

Epoch 1/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4301 - loss: 2.3657 - val_accuracy: 0.4847 - val_loss: 1.4380
Epoch 2/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4301 - loss: 1.3738 - val_accuracy: 0.4857 - val_loss: 0.8783
Epoch 3/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4153 - loss: 0.8718 - val_accuracy: 0.4659 - val_loss: 0.7472
Epoch 4/4
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4538 - loss: 0.7575 - val_accuracy: 0.4758 - val_loss: 0.7169


In [14]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [15]:
history = model_B_on_A.fit(x_train_B, y_train_B, epochs=16, validation_data=(x_valid_B, y_valid_B))

Epoch 1/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4999 - loss: 0.7136 - val_accuracy: 0.5668 - val_loss: 0.6670
Epoch 2/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6142 - loss: 0.6591 - val_accuracy: 0.6548 - val_loss: 0.6200
Epoch 3/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7168 - loss: 0.6096 - val_accuracy: 0.7221 - val_loss: 0.5807
Epoch 4/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7576 - loss: 0.5681 - val_accuracy: 0.7794 - val_loss: 0.5474
Epoch 5/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7910 - loss: 0.5329 - val_accuracy: 0.8200 - val_loss: 0.5190
Epoch 6/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8499 - loss: 0.5028 - val_accuracy: 0.8506 - val_loss: 0.4941
Epoch 7/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8545 - loss: 0.4764 - val_accuracy: 0.8675 - val_loss: 0.4720
Epoch 8/16
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8835 - loss: 0.4530 - val_accuracy: 0.8803 - val_loss: 0.4521
Epoch 9